# Hacker News: Analyzing News Posts to See What We Can Learn

In this project, we'll work with a dataset of submissions to popular technology site [Hacker News](https://news.ycombinator.com/). This is a website where users submit posts to receive votes and comments, very similarly to reddit. It's particularly popular in the IT world and startup circles - some posts get a few hundred thousand visitors as a result.

## Introduction

This is the [dataset](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts) we'll be working with, but using only a sample of the 300,000+ rows. After removing all submissions without comments and randomly sampling from the remainder, I'm left with about 20,000 rows of data.

In [1]:
# Pulling the data
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

# Taking a quick look
for row in hn[:5]:
    print(row)
    print()

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']

['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']

['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']

['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']



## Removing Headers from a List of Lists

The first row seems to be only headers and this isn't something I want factored in to results throughout the analysis, so I'll remove this and assign it to its own variable.

In [2]:
# Headers
headers = hn[:1]

# Removing headers from the data set
hn = hn[1:]

# Testing to make sure this worked
print('These are the headers:')
print(headers)
print()
print()
print('These are the first 5 rows of data:')
for row in hn[:5]:
    print(row)
    print()

These are the headers:
[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']]


These are the first 5 rows of data:
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']

['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']

['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']

['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']

['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0

## Extracting Ask HN and Show HN Posts

Within this dataset, I'm particularly interested in looking at posts that being with either "Ask HN" or "Show HN". When users want to ask the Hacker News community a specific question, they start the title of their post with "Ask HN". Likewise, when users want to show the community something interesting, they begin their posts' title with "Show HN". 

I'm curious to know which of these two get more comments on average. I'd also like to know if the time at which posts are added affects the number of comments they receive.

So that I can study each on their own, I'll split the dataset into 3 new lists: Ask Posts, Show Posts, and Other Posts.

In [3]:
# Creating the new lists
ask_posts = []
show_posts = []
other_posts = []

# Sorting the data into their new lists
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

# How many posts are in each list?
print('There are a total of ' + str(('{:,}'.format(len(ask_posts)))) + ' "Ask HN" posts.')
print('There are a total of ' + str(('{:,}'.format(len(show_posts)))) + ' "Show HN" posts.')
print('There are a total of ' + str(('{:,}'.format(len(other_posts)))) + ' other posts.')

There are a total of 1,744 "Ask HN" posts.
There are a total of 1,162 "Show HN" posts.
There are a total of 17,194 other posts.


## Calculating the Average Number of Commens for Ask HN and Show HN Posts

Now I can check to see if Ask posts or Show posts receive more comments on average.

In [4]:
# Finding the total number of comments for both Ask and Show posts
total_ask_comments = 0
total_show_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments

# Finding the average number of comments for both Ask and Show posts    
avg_ask_comments = total_ask_comments / len(ask_posts)
avg_show_comments = total_show_comments / len(show_posts)
print('Ask posts have an average of ' + str('{:.2f}'.format(avg_ask_comments)) + ' comments for each post')
print('Show posts have an average of ' + str('{:.2f}'.format(avg_show_comments)) + ' comments for each post')

Ask posts have an average of 14.04 comments for each post
Show posts have an average of 10.32 comments for each post


We can clearly see that Ask posts have an average of about 14 comments each, while Show posts are only averaging just over 10 comments each. This suggests that Ask posts inspire more conversation and/or input than Show posts do.

Because Ask posts are more likely to receive comments, I'll focus the remaining analysis just on these posts.

## Finding the Amount of Ask Posts and Comments by Hour Created

I'm curious to see if Ask posts created at a certain time of day are more likely to attract comments. Let's find out!

In [5]:
# Calculating the amount of ask posts created during each hour of day and the number of comments received.
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])]
    )

comments_by_hour = {}
counts_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for each_row in result_list:
    date = each_row[0]
    comment = each_row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1

comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

## Calculating the Average Number of Comments for Ask HN Posts by Hour

I've now created 2 dictionaries:
* __counts_by_hour__ contains the number of Ask posts created during each hour of the day.
* __comments_by_hour__ contains the corresponding number of comments ask posts created at each hour received.

Using these dictionaries, I'm going to calculate the average number of comments for posts created during each hour of the day.

In [6]:
# Calculating the average amount of comments `Ask HN` posts created at each hour of the day receive.
avg_by_hour = []

for hour in counts_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])
    
avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

## Sorting and Printing Values from a List of Lists

Although I now have the results I need, this format makes it difficult to identify the hours with the highest values. I'll finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [7]:
# Sorting the comment averages by most popular, in descending order
swap_avg_by_hour = []

for hour in avg_by_hour:
    swap_avg_by_hour.append([hour[1], hour[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [8]:
# Printing the results
print('Top 5 Hours for "Ask HN" Comments')
for avg, hr in sorted_swap[:5]:
    print(
        '{}: {:.2f} average comments per post'.format(
            dt.datetime.strptime(hr, '%H').strftime('%H:%M'),avg
        )
    )

Top 5 Hours for "Ask HN" Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

## Conclusion

In this project, I analyzed ask posts and show posts to determine which type of post and and what hour of posting will receive the most comments, on average. Based on this analysis, to maximize the amount of comments a post receives, I'd recommend the post be categorized as an ask post and created between 15:00 and 16:00 (3:00 pm EST - 4:00 pm EST).

However, it should be noted that the data set I analyzed excluded posts without any comments. Given that, it's more accurate to say that _of the posts that received comments_, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm EST - 4:00 pm EST) received the most comments on average.